In [109]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [110]:
# !pip install google-colab

In [111]:
import pandas as pd
import numpy as np
import os
import copy
import random
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
# from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers, Input
from tensorflow.keras.applications.resnet50 import ResNet50
# from tensorflow.keras.layers import *
from tensorflow.keras.layers import Dense, Dropout, LSTM, ZeroPadding2D, Conv2D, MaxPooling2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# from google.colab.patches import cv2_imshow

In [112]:
train=pd.read_csv('../input/facial-keypoints/training/training.csv')
test=pd.read_csv('../input/facial-keypoints/test/test.csv')


In [113]:
test.shape

In [114]:
test.head(10)

In [115]:
train.shape

In [116]:
train = train.dropna()

In [117]:
train.info()

In [118]:
train.head()

In [119]:
train['Image'] = train['Image'].apply(lambda x: np.fromstring(x, dtype = 'int', sep = ' ').reshape(96,96))

In [120]:
train.Image[0].shape

In [121]:
i = random.randint(1, len(train.Image))
print(i)
plt.imshow(train['Image'][i], cmap = 'gray')
for j in range(0,30,2):
    plt.plot(train.loc[i][j], train.loc[i][j+1], 'rx')
plt.show()

In [122]:
fig = plt.figure(figsize = (20,20))

for z in range(16):
    i = random.randint(1, len(train.Image))
    ax = plt.subplot(4,4, z+1)
    plt.imshow(train['Image'][i], cmap = 'gray')
    for j in range(0,30,2):
        plt.plot(train.loc[i][j], train.loc[i][j+1], 'rx')
plt.show()

In [123]:
train_copy = copy.copy(train)
columns = train_copy.columns[:-1]
columns

# DATA PREPARATION

In [124]:
train_copy['Image'] = train_copy['Image'].apply(lambda x: np.flip(x, axis = 1))
for i in range(len(columns)):
    if i%2 == 0:
        train_copy[columns[i]] = train_copy[columns[i]].apply(lambda x: 96. - float(x))

In [125]:
i = 2124
plt.imshow(train['Image'][i], cmap = 'gray')
for j in range(0,30,2):
    plt.plot(train.loc[i][j], train.loc[i][j+1], 'rx')
plt.show()

In [126]:
i = 2124
plt.imshow(train_copy['Image'][i], cmap = 'gray')
for j in range(0,30,2):
    plt.plot(train_copy.loc[i][j], train_copy.loc[i][j+1], 'rx')
plt.show()

In [127]:
df_aug = np.concatenate((train,train_copy))

In [128]:
df_aug.shape

In [129]:
train_copy = copy.copy(train)
train_copy['Image'] = train_copy['Image'].apply(lambda x: np.clip(random.uniform(1.5,2)*x, .0, 255.0))
df_aug = np.concatenate((df_aug, train_copy))
df_aug.shape

In [130]:
img = df_aug[:,-1]
img = img/255
X = np.empty((len(img),96,96,1))
print(X.shape)
for i in range(len(img)):
    X[i,] = np.expand_dims(img[i], axis = 2)

X = np.asarray(X).astype(np.float32)
 

In [131]:
y = df_aug[:,:-1]
y = np.asarray(y).astype(np.float32)
y.shape

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# MODEL

In [133]:
def res_block(X, f, stage):
    
    X_copy = X
    f1,f2,f3 = f

    #main path
    X = Conv2D(f1, (1,1), strides=(1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
    X = MaxPooling2D((2,2))(X)
    X = BatchNormalization(axis = 3,)(X)
    X = Activation('relu')(X)

    X = Conv2D(f2, (3,3), strides=(1,1), padding = 'same', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3,)(X)
    X = Activation('relu')(X)

    X = Conv2D(f3, (1,1), strides=(1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3,)(X)

    #Short path
    X_copy = Conv2D(f3, (1,1), strides=(1,1), kernel_initializer= glorot_uniform(seed = 0))(X_copy)
    X_copy = MaxPooling2D((2,2))(X_copy)
    X_copy = BatchNormalization(axis = 3,)(X_copy)
    
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    #Identity Block 1
    X_copy = X

    #main path
    X = Conv2D(f1, (1,1), strides=(1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3,)(X)
    X = Activation('relu')(X)

    X = Conv2D(f2, (3,3), strides=(1,1), padding = 'same', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3,)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(f3, (1,1), strides=(1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3,)(X)

    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    #Identity Block 2
    X_copy = X

    #main path
    X = Conv2D(f1, (1,1), strides=(1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3,)(X)
    X = Activation('relu')(X)

    X = Conv2D(f2, (3,3), strides=(1,1), padding = 'same', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3,)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(f3, (1,1), strides=(1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3,)(X)

    X = Add()([X,X_copy])
    X = Activation('relu')(X)
    
    return X 


In [134]:
input_shape = (96,96,1)

X_input = Input(input_shape)
X = ZeroPadding2D(padding=(3,3))(X_input)
#conv
X = Conv2D(64, (7,7), strides = (2,2), kernel_initializer= glorot_uniform(seed = 0))(X_input)
X = BatchNormalization(axis =3,)(X)
X = Activation('relu')(X)
X = Conv2D(64, (3,3), padding = 'same', kernel_initializer= glorot_uniform(seed = 0))(X_input)
X = BatchNormalization(axis =3,)(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides = (2,2))(X)


#res_block
X = res_block(X, [64,128,256], 2)
# #conv2D
# X = Conv2D(64, (3,3), padding = 'same', kernel_initializer= glorot_uniform(seed = 0))(X_input)
# X = BatchNormalization(axis =3,)(X)
# X = Activation('relu')(X)
# X = MaxPooling2D((2,2), strides = (2,2))(X)

#res_block
X = res_block(X, [128,256,512],3)

X = AveragePooling2D((2,2))(X)

X = Flatten()(X)
X = Dense(4096, activation = 'relu')(X)
X = Dropout(0.2)(X)
X = Dense(2048, activation = 'relu')(X)
X = Dropout(0.1)(X)
X_output = Dense(30, activation = 'relu')(X)

model1_ = Model(inputs = X_input, outputs = X_output)

In [135]:
model1_.summary()

In [136]:
adam = tf.keras.optimizers.Adam(learning_rate= 0.0001, beta_1= 0.9, beta_2= 0.999, amsgrad=False)
model1_.compile(loss = 'mean_squared_error', metrics = ['accuracy'], optimizer = adam)

In [137]:
checkpointer = ModelCheckpoint(filepath='facial_keypoints.hdf5', verbose = 1, save_best_only= True)
#callbacks = [checkpointer]

In [140]:
history = model1_.fit(X_train, y_train, batch_size = 32, epochs = 1000, validation_split = 0.05, )

In [141]:
model_json = model1_.to_json()
with open("FacialKey_Points.json","w") as json_file:
    json_file.write(model_json)

In [143]:
model1_.save_weights('./checkpoints/my_checkpoint')

In [ ]:
model1_